Script to calculate return periods of the observed water level data. Can either use the complete time series or the 2003-2023 period (i.e., GloFAS analysis period). 

In [1]:
# import relevant packages
import pandas as pd
import numpy as np
from scipy.stats import genextreme
import os

In [5]:
# define country and directory
country = 'zimbabwe'
directory = '/s3/scratch/jamie.towner/flood_aa'   
output_directory = os.path.join(directory, country, "outputs/thresholds")
os.makedirs(output_directory, exist_ok=True)  # create directory if it does not already exist 

In [6]:
# define paths to data
observed_data_directory = os.path.join(directory, country, "data/observations/gauging_stations/all_stations")
observed_data_file = "observations.csv" # use observations.csv for 2003-2023 period

# load data
observed_data_path = os.path.join(observed_data_directory, observed_data_file)
observed_data = pd.read_csv(observed_data_path)

In [7]:
# convert date columns to datetime
observed_data["date"] = pd.to_datetime(observed_data["date"], format='mixed')

In [8]:
# check data
observed_data

,date,ruware,malapati,mutirikwi,makwe,runde,beitbridge,manyuchi,kwalu,bangala,ingwesi,tokwane,tokwe
0,2003-01-01,1.21,NaN,0.20,0.0,0.75,0.0,0.1,NaN,256.02,0.0,1.28,0.53
1,2003-01-02,1.23,NaN,0.18,0.0,1.02,0.0,0.1,NaN,235.23,0.0,1.10,0.39
2,2003-01-03,1.17,NaN,0.18,0.0,1.01,0.0,0.1,NaN,231.20,0.0,0.98,0.71
3,2003-01-04,1.08,NaN,0.17,0.0,1.00,0.0,0.1,NaN,229.51,0.0,1.00,1.53
4,2003-01-05,1.09,NaN,0.16,0.0,1.00,0.0,0.1,NaN,227.82,0.0,1.03,1.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7665,2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7666,2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7667,2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7668,2023-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# function to calculate return periods for a given station's data
def calculate_return_periods(station_data, years=[2, 5, 10, 20]):
    # drop NA values
    station_data = station_data.dropna().copy()

    # extract the year from the date column 
    station_data.loc[:, 'year'] = station_data['date'].dt.year
    
    # group by year and get the maximum value for each year 
    annual_max = station_data.groupby('year')[station_data.columns[1]].max()
    
    # fit the data to a GEV distribution (Generalized Extreme Value distribution)
    #params = genextreme.fit(annual_max)

    # calculate the return period for each year 
    return_periods = {}
    for return_year in years:
        # the formula for return period is: 1 / (1 - F(x))
        # F(x) is the CDF of the fitted distribution at the threshold (max value)
        threshold = np.percentile(annual_max, 100 * (1 - 1/return_year))
        #threshold = genextreme.ppf(1 - 1/return_year, *params)
        return_periods[return_year] = threshold

    return return_periods

# initialize a dictionary to store return periods for each station
return_periods_dict = {}

# iterate over each station in the observed_data 
for station in observed_data.columns:
    if station == 'date':
        continue  # Skip 'date' column

    # get the data for this station
    station_data = observed_data[['date', station]]

    # skip if all values are NaN
    if station_data[station].dropna().empty:
        continue

    # calculate return periods for the station
    return_periods = calculate_return_periods(station_data)
    
    # store the return periods in the dictionary
    return_periods_dict[station] = return_periods

# convert the dictionary to a dataframe
return_periods_df = pd.DataFrame.from_dict(return_periods_dict, orient='index')

In [11]:
# check the output
return_periods_df

,2,5,10,20
ruware,26.410,87.808,133.668,163.2920
mutirikwi,50.970,108.250,158.410,232.3900
makwe,12.020,18.030,29.980,34.8725
runde,169.100,481.672,555.208,589.9605
beitbridge,951.390,2425.424,2755.428,2993.6320
manyuchi,0.740,101.974,112.482,130.3980
bangala,346.090,473.762,490.428,548.7990
ingwesi,0.000,0.000,2.535,7.8200
tokwane,121.355,463.980,752.120,992.6525
tokwe,802.910,1496.578,1759.359,1968.7915


In [ ]:
# save output as a csv 
return_periods_df.to_csv(os.path.join(output_directory, "return_periods.csv"), index=False)